In [1]:
import sys
# appending a path
sys.path.append('../')

import torch
import torch.nn as nn
import torch.nn.functional as F

import pychop
from pychop import Chopi
pychop.backend('torch')

class QuantizedNet(nn.Module):
    def __init__(self):
        super(QuantizedNet, self).__init__()
        # Convolutional Layers
        self.conv1d = nn.Conv1d(1, 16, 3, padding=1)
        self.conv2d = nn.Conv2d(1, 16, 3, padding=1)
        self.conv3d = nn.Conv3d(1, 16, 3, padding=1)

        self.wquant_conv1d = Chopi(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_conv2d = Chopi(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_conv3d = Chopi(8, symmetric=True, per_channel=True, channel_dim=0)

        # Recurrent Layers
        self.lstm = nn.LSTM(16, 32, batch_first=True)
        self.gru = nn.GRU(32, 16, batch_first=True)

        self.wquant_lstm_ih = Chopi(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_lstm_hh = Chopi(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_gru_ih = Chopi(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_gru_hh = Chopi(8, symmetric=True, per_channel=True, channel_dim=0)

        # BatchNorm, Pooling, and Linear
        self.bn2d = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(16 * 14 * 14, 10)  # 14x14 after pooling
        self.wquant_fc = Chopi(8, symmetric=True, per_channel=True, channel_dim=0)

        # Activation Chopis
        self.aquant_conv1d = Chopi(8, symmetric=False, per_channel=True, channel_dim=1)
        self.aquant_conv2d = Chopi(8, symmetric=False, per_channel=True, channel_dim=1)
        self.aquant_conv3d = Chopi(8, symmetric=False, per_channel=True, channel_dim=1)
        self.aquant_lstm = Chopi(8, symmetric=False, per_channel=True, channel_dim=2)  # dim 2 for [batch, seq, feat]
        self.aquant_gru = Chopi(8, symmetric=False, per_channel=True, channel_dim=2)
        self.aquant_fc = Chopi(8, symmetric=False)

        self.relu = nn.ReLU()

    def forward(self, x_1d, x_2d, x_3d, x_seq, training=True):
        # Conv1d
        w1d = self.wquant_conv1d(self.conv1d.weight, training=training)

        x_1d = F.conv1d(x_1d, w1d, self.conv1d.bias, padding=1)
        x_1d = self.aquant_conv1d(x_1d, training=training)
        x_1d = self.relu(x_1d)

        # Conv2d
        w2d = self.wquant_conv2d(self.conv2d.weight, training=training)
        x_2d = F.conv2d(x_2d, w2d, self.conv2d.bias, padding=1)
        x_2d = self.bn2d(x_2d)
        x_2d = self.aquant_conv2d(x_2d, training=training)
        x_2d = self.relu(x_2d)
        x_2d = self.pool(x_2d)  # [2, 16, 14, 14]
        x_2d = x_2d.view(x_2d.size(0), -1)  # [2, 16*14*14]

        # Conv3d
        w3d = self.wquant_conv3d(self.conv3d.weight, training=training)
        x_3d = F.conv3d(x_3d, w3d, self.conv3d.bias, padding=1)
        x_3d = self.aquant_conv3d(x_3d, training=training)
        x_3d = self.relu(x_3d)

        # RNN (LSTM + GRU)
        # Note: PyTorch RNNs use fused ops, so we quantize weights but apply them manually is complex.
        # For simplicity, we quantize inputs/outputs here; true integer RNNs need custom ops.
        w_lstm_ih = self.wquant_lstm_ih(self.lstm.weight_ih_l0, training=training)
        w_lstm_hh = self.wquant_lstm_hh(self.lstm.weight_hh_l0, training=training)
        x_seq, _ = self.lstm(x_seq)  # Fused op, weights not directly applied here
        x_seq = self.aquant_lstm(x_seq, training=training)
        w_gru_ih = self.wquant_gru_ih(self.gru.weight_ih_l0, training=training)
        w_gru_hh = self.wquant_gru_hh(self.gru.weight_hh_l0, training=training)
        x_seq, _ = self.gru(x_seq)
        x_seq = self.aquant_gru(x_seq, training=training)
        x_seq = x_seq[:, -1, :]  # Last timestep

        # Linear
        w_fc = self.wquant_fc(self.fc.weight, training=training)
        x_2d = F.linear(x_2d, w_fc, self.fc.bias)  # [2, 3136] * [3136, 10]
        x_2d = self.aquant_fc(x_2d, training=training)

        return x_1d, x_2d, x_3d, x_seq


In [2]:
if __name__ == "__main__":
    model = QuantizedNet()
    model.train()

    x_1d = torch.randn(2, 1, 64)  # [batch, channels, length]
    x_2d = torch.randn(2, 1, 28, 28)  # [batch, channels, height, width]
    x_3d = torch.randn(2, 1, 16, 16, 16)  # [batch, channels, depth, height, width]
    x_seq = torch.randn(2, 10, 16)  # [batch, seq_len, features]

    # Training mode
    out_1d, out_2d, out_3d, out_seq = model(x_1d, x_2d, x_3d, x_seq, training=True)
    print("Training outputs:")
    print("Conv1d:", out_1d.shape, out_1d[0, 0, :5])
    print("Conv2d+FC:", out_2d.shape, out_2d[0, :5])
    print("Conv3d:", out_3d.shape, out_3d[0, 0, 0, 0, :5])
    print("LSTM+GRU:", out_seq.shape, out_seq[0, :5])

    # Inference mode
    model.eval()
    with torch.no_grad():
        out_1d, out_2d, out_3d, out_seq = model(x_1d, x_2d, x_3d, x_seq, training=False)
        print("\nInference outputs (INT8):")
        print("Conv1d:", out_1d.shape, out_1d[0, 0, :5])
        print("Conv2d+FC:", out_2d.shape, out_2d[0, :5])
        print("Conv3d:", out_3d.shape, out_3d[0, 0, 0, :5])
        print("LSTM+GRU:", out_seq.shape, out_seq[0, :5])


Training outputs:
Conv1d: torch.Size([2, 16, 64]) tensor([0.0000, 0.1261, 0.1855, 0.0000, 0.5060])
Conv2d+FC: torch.Size([2, 10]) tensor([ 0.5681, -0.1191, -0.4265, -0.8062,  0.1160])
Conv3d: torch.Size([2, 16, 16, 16, 16]) tensor([0.0985, 0.0000, 0.0000, 0.4210, 0.0000])
LSTM+GRU: torch.Size([2, 16]) tensor([-0.2229, -0.1467, -0.0997,  0.1245,  0.3705])

Inference outputs (INT8):
Conv1d: torch.Size([2, 16, 64]) tensor([46., 57., 62., 36., 89.])
Conv2d+FC: torch.Size([2, 10]) tensor([70., 72., 24.,  7., 93.])
Conv3d: torch.Size([2, 16, 16, 16, 16]) tensor([[67., 62., 46., 77., 55., 66., 59., 75., 65., 68., 64., 65., 36., 93.,
         70., 65.],
        [57., 56., 84., 53., 68., 62., 75., 56., 51., 64., 51., 56., 88., 43.,
         64., 54.],
        [52., 70., 45., 74., 61., 65., 73., 45., 57., 45., 82., 41., 56., 58.,
         56., 60.],
        [68., 51., 70., 64., 84., 48., 31., 74., 56., 83., 50., 79., 50., 59.,
         71., 66.],
        [55., 77., 61., 63., 61., 45., 56., 58., 

In [1]:

import sys
# appending a path
sys.path.append('../')

import torch
import torch.nn as nn
import torch.nn.functional as F

import pychop
from pychop.layers import IntQuantizedLayer # Without the need to specify pychop.backend('torch')

from pychop import Chopi

class QuantizedNet2(nn.Module):
    def __init__(self):
        super(QuantizedNet2, self).__init__()
        # Convolutional Layers
        self.conv1d = nn.Conv1d(1, 16, 3, padding=1)
        self.conv2d = nn.Conv2d(1, 16, 3, padding=1)
        self.conv3d = nn.Conv3d(1, 16, 3, padding=1)

        self.wquant_conv1d = IntQuantizedLayer(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_conv2d = IntQuantizedLayer(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_conv3d = IntQuantizedLayer(8, symmetric=True, per_channel=True, channel_dim=0)

        # Recurrent Layers
        self.lstm = nn.LSTM(16, 32, batch_first=True)
        self.gru = nn.GRU(32, 16, batch_first=True)

        self.wquant_lstm_ih = IntQuantizedLayer(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_lstm_hh = IntQuantizedLayer(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_gru_ih = IntQuantizedLayer(8, symmetric=True, per_channel=True, channel_dim=0)
        self.wquant_gru_hh = IntQuantizedLayer(8, symmetric=True, per_channel=True, channel_dim=0)

        # BatchNorm, Pooling, and Linear
        self.bn2d = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(16 * 14 * 14, 10)  # 14x14 after pooling
        self.wquant_fc = IntQuantizedLayer(8, symmetric=True, per_channel=True, channel_dim=0)

        # Activation IntQuantizedLayers
        self.aquant_conv1d = IntQuantizedLayer(8, symmetric=False, per_channel=True, channel_dim=1)
        self.aquant_conv2d = IntQuantizedLayer(8, symmetric=False, per_channel=True, channel_dim=1)
        self.aquant_conv3d = IntQuantizedLayer(8, symmetric=False, per_channel=True, channel_dim=1)
        self.aquant_lstm = IntQuantizedLayer(8, symmetric=False, per_channel=True, channel_dim=2)  # dim 2 for [batch, seq, feat]
        self.aquant_gru = IntQuantizedLayer(8, symmetric=False, per_channel=True, channel_dim=2)
        self.aquant_fc = IntQuantizedLayer(8, symmetric=False)

        self.relu = nn.ReLU()

    def forward(self, x_1d, x_2d, x_3d, x_seq):
        w1d = self.wquant_conv1d(self.conv1d.weight)

        x_1d = F.conv1d(x_1d, w1d, self.conv1d.bias, padding=1)
        x_1d = self.aquant_conv1d(x_1d)
        x_1d = self.relu(x_1d)

        w2d = self.wquant_conv2d(self.conv2d.weight)
        x_2d = F.conv2d(x_2d, w2d, self.conv2d.bias, padding=1)
        x_2d = self.bn2d(x_2d)
        x_2d = self.aquant_conv2d(x_2d)
        x_2d = self.relu(x_2d)
        x_2d = self.pool(x_2d)  # [2, 16, 14, 14]
        x_2d = x_2d.view(x_2d.size(0), -1)  # [2, 16*14*14]

        w3d = self.wquant_conv3d(self.conv3d.weight)
        x_3d = F.conv3d(x_3d, w3d, self.conv3d.bias, padding=1)
        x_3d = self.aquant_conv3d(x_3d)
        x_3d = self.relu(x_3d)

        w_lstm_ih = self.wquant_lstm_ih(self.lstm.weight_ih_l0)
        w_lstm_hh = self.wquant_lstm_hh(self.lstm.weight_hh_l0)
        x_seq, _ = self.lstm(x_seq)  # Fused op, weights not directly applied here
        x_seq = self.aquant_lstm(x_seq)
        w_gru_ih = self.wquant_gru_ih(self.gru.weight_ih_l0)
        w_gru_hh = self.wquant_gru_hh(self.gru.weight_hh_l0)
        x_seq, _ = self.gru(x_seq)
        x_seq = self.aquant_gru(x_seq)
        x_seq = x_seq[:, -1, :]  # Last timestep

        # Linear
        w_fc = self.wquant_fc(self.fc.weight)
        x_2d = F.linear(x_2d, w_fc, self.fc.bias)  # [2, 3136] * [3136, 10]
        x_2d = self.aquant_fc(x_2d)

        return x_1d, x_2d, x_3d, x_seq


In [2]:

if __name__ == "__main__":
    model = QuantizedNet2()
    model.train()

    x_1d = torch.randn(2, 1, 64)  # [batch, channels, length]
    x_2d = torch.randn(2, 1, 28, 28)  # [batch, channels, height, width]
    x_3d = torch.randn(2, 1, 16, 16, 16)  # [batch, channels, depth, height, width]
    x_seq = torch.randn(2, 10, 16)  # [batch, seq_len, features]

    # Training mode
    out_1d, out_2d, out_3d, out_seq = model(x_1d, x_2d, x_3d, x_seq)
    print("Training outputs:")
    print("Conv1d:", out_1d.shape, out_1d[0, 0, :5])
    print("Conv2d+FC:", out_2d.shape, out_2d[0, :5])
    print("Conv3d:", out_3d.shape, out_3d[0, 0, 0, 0, :5])
    print("LSTM+GRU:", out_seq.shape, out_seq[0, :5])

    # Inference mode
    model.eval()
    with torch.no_grad():
        out_1d, out_2d, out_3d, out_seq = model(x_1d, x_2d, x_3d, x_seq)
        print("\nInference outputs (INT8):")
        print("Conv1d:", out_1d.shape, out_1d[0, 0, :5])
        print("Conv2d+FC:", out_2d.shape, out_2d[0, :5])
        print("Conv3d:", out_3d.shape, out_3d[0, 0, 0, :5])
        print("LSTM+GRU:", out_seq.shape, out_seq[0, :5])


Training outputs:
Conv1d: torch.Size([2, 16, 64]) tensor([89., 48., 12., 45., 42.])
Conv2d+FC: torch.Size([2, 10]) tensor([54., 25.,  0., 39., 42.])
Conv3d: torch.Size([2, 16, 16, 16, 16]) tensor([80., 87., 60., 43., 58.])
LSTM+GRU: torch.Size([2, 16]) tensor([  0.,   0., 127.,   0.,   0.])

Inference outputs (INT8):
Conv1d: torch.Size([2, 16, 64]) tensor([89., 48., 12., 45., 42.])
Conv2d+FC: torch.Size([2, 10]) tensor([54., 25.,  0., 39., 42.])
Conv3d: torch.Size([2, 16, 16, 16, 16]) tensor([[ 80.,  87.,  60.,  43.,  58.,  46.,  44.,  48.,  53.,  69.,  69.,  66.,
          62.,  57.,  66.,  80.],
        [ 68.,  32., 103.,  68.,  62.,  73.,  94.,  79.,  65.,  46.,  63.,  98.,
          61.,  52.,  61.,  78.],
        [ 51.,  56., 102.,  79.,  62.,  56.,  37.,  54.,  62.,  76.,  67.,  19.,
          71.,  72.,  53.,  88.],
        [ 69.,  67.,  78.,  63.,  63.,  40.,  28.,  47.,  64.,  84.,  58.,  59.,
          74.,  69.,  60., 100.],
        [ 28.,  48.,  46.,  46.,  38.,  82.,  48.,